## 1. Algoritmos

Temas:
- Aproximación a $e$
- Shooting algorithm

### Introducción

Un algoritmo es una serie de instrucciones que permiten resolver un problema. 

- Conjunto finito de pasos
- Instrucciones concretas y bien definidas
- Con un punto de partida y un punto de llegada

### Aproximación a $e$

El número $e$ puede expresarse como

$$
e = \sum_{n = 0}^\infty \frac{1}{n!}
$$

Dado que no podemos sumar infinitos términos, podemos aproximarnos al número $e$ con la suma de los primeros $N$ términos.

El algoritmo para aproximarnos a $e$ sería

1. **Inicialización**:
   - Establecer la tolerancia `tol` (el criterio para parar de sumar, cuando la aproximación está "suficientemente cerca" del valor real de $e$) 
   - Establecer el número máximo de términos a sumar `N`
   - Inicializar la suma `e` a 1 (corresponde al primer término de la serie).
   - Inicializar el valor del factorial `fct` a 1 (inicialmente representa 0!).

2. **Iteración**:
   - Para $n$ desde 1 hasta `N`, hacer lo siguiente:
     - Calcular el factorial de $n$ acumulando en `fct` (`fct = fct * n`).
     - Actualizar el valor de `e` sumando $\frac{1}{fct} $ al valor actual de `e`.
     - Mostrar los valores actuales de `fct` y `e` para depuración.
     - Comparar la diferencia absoluta entre `e` y el valor real de $e$ (`exp(1)`):
       - Si la diferencia es menor o igual a `tol`, entonces:
         - Terminar el bucle (`break`).

3. **Resultado**:
   - El valor de `e` ahora contiene la aproximación de $e$ con la precisión especificada.

Este algoritmo se puede utilizar para aproximar el valor de $e$ con un error menor o igual a la tolerancia especificada, siempre que el número de iteraciones sea suficiente.

In [ ]:
# Definimos parámetros para el problema
tol = 0.001 # tolerancia (cuando es que estamos "suficientemente cerca")
N = 10000 # máximo número de iteraciones

# Valores iniciales
e = 1
fct = 1

for n = 1:N
	fct *= n # calculo el factorial en cada iteración
    e += 1 / fct # redefino e con cada loop
	@show n, fct, e
    if (abs(e - exp(1)) <= tol) # comparo con exp(1), el valor real
        break # si la aproximación está más cerca de e que tol, entonces corto el loop
    end
end

Para reiterar y aclarar que hicimos:

- `N` define el número máximo de iteraciones (para que el programa no corra infinitamente)
- `e` es nuestra aproximación de $e$, actualizado en cada iteración.
	- `e` empieza en 1 porque $0! = 1$ y la sumatoria empieza en $n=0$
- `fct` corresponde a `n!`, calculado manualmente en cada iteración. 
	- También podríamos haber utilizado la función `factorial(n)`
- `@show` es un macro, equivalente en este caso a: `println("(fct, e) = ($fct, $e)")

Podemos resolver el mismo problema con una función, permitiéndonos cambiar los parámetros de tolerancia y máximas iteraciones, sin reescribir el loop constantemente.

In [ ]:
function approx_e(tol; N = 10000)
	e = 0.0
	for n = 0:N
	    e += 1 / factorial(n)
		@show n, e
	    if (abs(e - exp(1)) <= tol)
	        break
	    end
	end
	return e
end

approx_e(0.0000000001)

### Shooting algorithm

#### El modelo

Partamos de un modelo de crecimiento neoclásico en horizonte finito:

$$
\max_{\{c_{t}, k_{t+1}\}_{t=0}^{T}} \sum_{t=0}^{T}\beta^{t}u(c_{t})
$$

Sujeto a las siguientes restricciones:

- Restricción de recursos: $\quad c_{t}+k_{t+1} \leq f(k_{t})\equiv F(k_{t}) + (1-\delta)k_{t}$
- Restricción de no-negatividad: $\quad c_t \geq 0, \quad k_{t+1} \geq 0$
- Condición inicial: $\quad k_0 > 0$ dado

La ecuación de Euler del modelo viene dada por:

$$\text{(EE):}\quad u'(c_t) = \beta u'(c_{t+1}) f'(k_{t+1})$$

#### El algoritmo

Si no hay depreciación completa del capital ($\delta < 1$), entonces no hay solución en forma cerrada. Para hallar una solución, usamos un algoritmo computacional.

El **shooting algorithm** involucra los siguientes pasos:

1. Adivinar un valor inicial para $c_0$
2. Computar la secuencia que cumpla la Ecuación de Euler $\forall t$ dado $(k_0, c_0)$.
3. Testear si la secuencia es la solución, chequeando que se cumple la condición terminal sobre $k_{T+1}$
	- Si $k_{T+1} \approx 0$ entonces:
		- La secuencia que empieza con $(k_{0}, c_{0})$ es la solución
	- Si no, elegir un nuevo valor de $c_0$ y repetir pasos 1-3

Dos problemas a resolver en la implementación:

- ¿Cómo elegir el valor inicial?  Usar solución para $\delta = 1$
- ¿Cómo actualizar el valor de $c_0$? Algoritmos de búsqueda (ej: bisection search)

#### La implementación

Para implementar el algoritmo tenemos que suponer una forma para la función de utilidad y la función de producción, el valor de los principales parámetros del problema, y un horizonte temporal para resolver. 

En este caso, vamos a suponer que la función de utilidad es logarítimica y que la función de producción es una Cobb-Douglas.

$$
u(c) = \log(c); \quad F(k) = Ak^\alpha
$$

Dado que $u'(c)>0$ la restricción de recursos siempre va a estar activa:

$$
c_t + k_{t+1} = \underbrace{F(k_t) - (1-\delta)k_t}_{f(k_t)}
$$

y podemos expresar la EE como

$$
\frac{1}{f(k_t) - k_{t+1}} = \beta \cdot \frac{1}{f(k_{t+1}) + k_{t+2}} \cdot f'(k_{t+1})
$$

Comenzamos por inicializar los parámetros del modelo. Supondremos que $k_0 = 10, \beta = 0.95, \delta = 0.1, A = 1, \alpha = 0.3$. 

In [ ]:
k0 = 10.0
β = 0.95
δ = 0.1
α = 0.3

A = 1.0

Luego, tenemos que encontrar una forma de despejar la trayectoria del capital y el consumo para que la secuencia cumpla la EE en cada punto. 

Despejando de la EE tenemos que 
$$c_{t+1} = u'^{-1}\left(\frac{u'(c_{t})}{\beta \cdot f'(k_{t+1})}\right)$$

donde
$$\begin{array}{rl}
f(k) &= k^\alpha + (1-\delta)k \\ 
f'(k)&= \alpha k^{\alpha -1} + 1 - \delta \\
u(c) &= log(c) \\
u'(c)&= 1/c \\
u'^{-1}(c) &= 1/c
\end{array}$$

Definimos las funciones principales del problema:

In [ ]:
"Utility function"
u(c) = log(c)

"Derivative of utility"
u_prime(c) = 1/c

"Inverse of derivative of utility"
u_prime_inv(c) = 1/c

"Production function"
function F(k, A = A, α = α)
    if k > 0
        return A * k^α
    else
        return 0
    end
end

"Derivative of production function"
function F_prime(k, A = A, α = α) 
    if k < 0
        return 0
    else
        return A * α * k^(α-1)
    end
end

"Production function (w depreciation)"
function f(k, A = A, α = α, δ = δ)
    if k > 0
        return F(k) + (1 - δ) * k
    else
        return 0
    end
end

"Derivative of production function (w depreciation)"
function f_prime(k, A = A, α = α, δ = δ) 
    if k < 0
        return 0
    else
        return F_prime(k) + (1 - δ)
    end
end

Creamos una función que devuelva $(k_{t+1}, c_{t+1})$ en base a $(k_t, c_t)$

In [ ]:
"Compute next capital and consumption based on current values"
function next_k_c(k, c, A = A, α = α, δ = δ, β = β)
    k_next = f(k) - c
    c_next = u_prime_inv(u_prime(c) / (β * (f_prime(k_next))))
    return k_next, c_next
end

Por último, creamos una función `shooting` que nos devuelve toda la secuencia de consumo y capital, en base a los valores inciales de $(k_0, c_0)$, siguiendo la regla establecida en `next_k_c`.

In [ ]:
"Shooting algorithm"
function shooting(k0, c0, T, A = A, α = α, δ = δ, β = β; verbose = false)
    # initialize vectors 
    k_vec = zeros(T+2)
    c_vec = zeros(T+1)
    
    k_vec[1] = k0
    c_vec[1] = c0

    for t in 1:T
        k_vec[t+1], c_vec[t+1] = next_k_c(k_vec[t], c_vec[t], A, α, δ, β)
        if verbose
            display("$t : k = $(k_vec[t+1]), c = $(c_vec[t+1])")
        end
    end

    k_vec[T+2] = f(k_vec[T+1]) + (1 - δ) * k_vec[T+1] - c_vec[T+1]

    return [k_vec, c_vec]
end;

Implementamos un algoritmo de búsqueda por bisección:

1. Determinamos rango para buscar $c_0 = (c_{lo}, c_{hi})$ 
	- Tomamos dos valores extremos: 0 (maximo ahorro) y $f(k_0)$ (máximo consumo)
2. Testear el punto medio del intervalo de búsqueda como posible $c_0$
3. Computar secuencias de capital y consumo que cumplan con la EE en cada período
4. Chequear si $k_{T+1} = 0$
	- Si $k_{T+1} \approx 0$, el valor probado para $c_0$ es óptimo.
	- Si $k_{T+1} > 0$ hay demasiado ahorro. Elegir $c_0$ más alto (descartar valores por debajo de $c_0$).
	- Si $k_{T+1} < 0$ hay demasiado consumo. Elegir $c_0$ más bajo (descartar valores por encima de $c_0$).
5. Repetir pasos 2–4 hasta hallar el óptimo.

In [ ]:
k0 = 10.0

c0_lo = 0.0   # mínimo: no hay consumo
c0_hi = f(k0) # máximo: no hay ahorro

c0 = (c0_lo + c0_hi) / 2 # guess inicial: punto medio

maxiter = 10e5           # evitar loop infinito
tol = 10e-13             # tolerancia

i = 0
while i < maxiter
	c0 = (c0_lo + c0_hi) / 2     # actualizar guess
	k,c = shooting(k0, c0, 100)  # calculo vectores de k y c en base a guess

    if k[end] ≈ 0 || (c0_hi - c0_lo) / 2 < tol
        display("Solución: c0 = $(c0)")
        break
    elseif k[end] > 0
        c0_lo = c0
    else
        c0_hi = c0
    end

	i += 1
end
c0_optim = c0;

Por último, graficamos las trayectorias óptimas de capital y consumo:

In [ ]:
using LaTeXStrings, Plots

In [ ]:
# plot optimal trajectories of capital and consumption
k_vec, c_vec = shooting(k0, c0_optim, 100);
plotk = plot(k_vec, title = "Capital", yaxis = L"k_t")
plotc = plot(c_vec, title = "Consumption", yaxis = L"c_t",  color = "red")
plot(plotk, plotc, layout = (1, 2), legend = false, xaxis = "t")